# Per Bounding Box Luminosity Calculation

This notebook demonstrates how to calculate the luminosity of images and their respective bounding boxes.
We will write a new table combining the columns of the input table with the calculated luminosity properties.

<div style="background-color: #e7f3fe; padding: 10px; border-left: 6px solid #2196F3; margin-bottom: 15px; color: #333;">
    <strong>Info:</strong> This notebook demonstrates a technique for adding columns to an existing Table.
    While this is a useful technique, there are some drawbacks to this approach:
    <ul>
        <li>The new Table will not be part of the lineage of the input Table.</li>
        <li>The new Table will contain a literal copy of all data in the input Table</li>
    </ul>
    In a future release of 3LC, adding columns to an existing Table will be supported natively.
</div>


In [ ]:
PROJECT_NAME = "Luminosity"
DATASET_NAME = "COCO128"
INSTALL_DEPENDENCIES = False
TEST_DATA_PATH = "../tests/test_data/data"
TLC_PUBLIC_EXAMPLES_DEVELOPER_MODE = True

In [ ]:
%%capture
if INSTALL_DEPENDENCIES:
    %pip --quiet install torch --index-url https://download.pytorch.org/whl/cu118
    %pip --quiet install torchvision --index-url https://download.pytorch.org/whl/cu118
    %pip --quiet install tlc

In [ ]:
from __future__ import annotations

from io import BytesIO

import tqdm
import numpy as np
from PIL import Image

import tlc

## Set Up Input Table

We will use a `TableFromCoco` to load the input dataset from a annotations file and a folder of images.

In [ ]:
table_url = tlc.Table.default_write_location() / f"{PROJECT_NAME}/table_from_coco.json"

annotations_file = tlc.Url(TEST_DATA_PATH  + "/coco128/annotations.json").to_absolute()
images_dir = tlc.Url(TEST_DATA_PATH + "/coco128/images").to_absolute()

input_table = tlc.TableFromCoco(
    url=table_url,
    dataset_name=DATASET_NAME,
    project_name=PROJECT_NAME,
    input_url=annotations_file.to_relative(),
    image_folder_url=images_dir.to_relative(),
    row_cache_url="../table_from_coco.parquet",
)

input_table.ensure_fully_defined()

## Calculate the Luminosity of Images and Bounding Boxes

In this section, we will calculate the luminosity property for each image as well as for each bounding box within the images.

We build the variables `per_image_luminosity` and `per_bb_luminosity` to store the luminosity properties for each image and bounding box, respectively.

In [ ]:
def calculate_luminosity(image: Image) -> float:
    np_image = np.array(image)
    axes_to_reduce = tuple(range(np_image.ndim - 1))
    avg_luminosity = np.mean(np_image, axis=axes_to_reduce) / 255.0
    return float(np.mean(avg_luminosity))

In [ ]:
per_bb_luminosity: list[list[float]] = []
per_image_luminosity: list[float] = []

bb_schema = input_table.row_schema.values["bbs"].values["bb_list"]

for row in tqdm.tqdm(input_table.table_rows, total=len(input_table), desc="Calculating luminosity"):
    image_filename = row["image"]
    image_bbs = row["bbs"]["bb_list"]

    image_bytes = tlc.Url(image_filename).read()
    image = Image.open(BytesIO(image_bytes))

    image_luminosity = calculate_luminosity(image)
    per_image_luminosity.append(image_luminosity)

    bb_luminosity_list: list[float] = []
    h, w = image.size

    for bb in image_bbs:
        bb_crop = tlc.BBCropInterface.crop(image, bb, bb_schema)
        bb_luminosity = calculate_luminosity(bb_crop)
        bb_luminosity_list.append(bb_luminosity)

    per_bb_luminosity.append(bb_luminosity_list)

## Create new Table containing luminosity properties

After calculating the luminosity, we will create a new table using a `TableWriter`.

### Setup the Schema of the output Table

In [ ]:
# Each entry in the list is a list of luminosity values for each bounding box in the image 
per_bb_luminosity_schema = tlc.Schema(
    value=tlc.Float32Value(
        value_min=0,
        value_max=1,
        number_role=tlc.NUMBER_ROLE_FRACTION,
    ),
    size0=tlc.DimensionNumericValue(value_min=0, value_max=1000),  # Max 1000 bounding boxes
    sample_type="hidden", # Hide this column when iterating over the "sample view" of the table
    writable=False,
)

per_image_luminosity_schema = tlc.Schema(
    value=tlc.Float32Value(
        value_min=0,
        value_max=1,
        number_role=tlc.NUMBER_ROLE_FRACTION,
    ),
    sample_type="hidden", # Hide this column when iterating over the "sample view" of the table
    writable=False,
)

schemas = {
    "per_bb_luminosity": per_bb_luminosity_schema,
    "per_image_luminosity": per_image_luminosity_schema,
}
schemas.update(input_table.row_schema.values)  # Copy over the schema from the input table

### Write the output Table

We will use a `TableWriter` to write the output table as a `TableFromParquet`.

In [ ]:
from tlc.core.metrics_writer.table_writer import TableWriter
from collections import defaultdict

table_writer = TableWriter(
    root_url=input_table.url.parent,
    base_name="added_luminosity_metrics",
    override_column_schemas=schemas,
)

# TableWriter accepts data as a dictionary of column names to lists
data = defaultdict(list)

# Copy over all rows from the input table
for row in input_table.table_rows:
    for column_name, column_value in row.items():
        data[column_name].append(column_value)

# Add the luminosity metrics
data["per_image_luminosity"] = per_image_luminosity
data["per_bb_luminosity"] = per_bb_luminosity

table_writer.add_batch(data)
table_writer.flush()

infos = table_writer.get_written_table_infos()
new_table = tlc.Table.from_url(infos[0]["url"])

### Inspect the properties of the output Table

In [ ]:
print(len(new_table))
print(new_table.columns)
print(new_table.url.to_relative(input_table.url))

Let's check which columns are present in the sample view / table view of the input and output tables:

In [ ]:
# Sample view of input table
input_table[0].keys()

In [ ]:
# Table view of input table
input_table.table_rows[0].keys()

In [ ]:
# Sample view of output table (does not contain the luminosity columns due to the sample_type="hidden" flag)
new_table[0].keys()

In [ ]:
# Table view of output table (contains the luminosity columns)
new_table.table_rows[0].keys()